## reading in data and importing things

In [2]:
#importing things
import pandas as pd
import os
import numpy as np
import csv
import pylab as pl
import matplotlib.pyplot as plt
import copy
import scipy
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import t
from scipy.stats import chi2
from scipy.stats import sem
from scipy.linalg import hadamard
from scipy.stats import ttest_ind_from_stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind_from_stats
from sklearn.manifold import TSNE
from collections import OrderedDict
import seaborn as sns
from functools import reduce
from scipy.optimize import curve_fit
from operator import itemgetter
import math

In [2]:
#reading in counts
rootdir = 'expression_reads/9114i'
list_of_dataframes = []
for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if '9114' in file:
                filepath = str(rootdir)+'/'+str(file)
                filename, file_extension = os.path.splitext(file)
                print(filepath)
                list_of_dataframes.append(pd.read_table(filepath,delimiter='\t\t',names=['variant',filename],index_col=None,dtype={'variant': str,'LogEC50':float}))

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['variant'],
                                            how='outer'), list_of_dataframes)
df_merged.fillna(0, inplace=True)
pd.DataFrame.to_csv(df_merged, 'expression_analysis/9114i_merged_reads.txt', sep=',', na_rep=float(0), index=False)
df_merged

expression_reads/9114i/counts_9114_i_F_1.txt


/Users/angelaphillips/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


expression_reads/9114i/counts_9114_i_F_3.txt
expression_reads/9114i/counts_9114_i_F_2.txt
expression_reads/9114i/counts_9114_i_F_6.txt
expression_reads/9114i/counts_9114_i_F_7.txt
expression_reads/9114i/counts_9114_i_F_5.txt
expression_reads/9114i/counts_9114_i_F_4.txt
expression_reads/9114i/counts_9114_i_F_8.txt


,variant,counts_9114_i_F_1,counts_9114_i_F_3,counts_9114_i_F_2,counts_9114_i_F_6,counts_9114_i_F_7,counts_9114_i_F_5,counts_9114_i_F_4,counts_9114_i_F_8
0,1100111101100100,1390.0,594.0,196.0,2610.0,2202.0,2350.0,2022.0,782.0
1,1111101010000111,1330.0,290.0,129.0,1399.0,1348.0,1409.0,691.0,1941.0
2,1000110100111000,1238.0,714.0,232.0,1608.0,2153.0,2072.0,923.0,1681.0
3,1101111000111101,1147.0,23.0,5.0,23.0,59.0,72.0,21.0,90.0
4,1100010101101011,1127.0,184.0,187.0,544.0,425.0,689.0,521.0,325.0
...,...,...,...,...,...,...,...,...,...
65531,0000100011010000,0.0,0.0,3.0,0.0,2.0,3.0,3.0,0.0
65532,1000100000111110,0.0,0.0,2.0,3.0,26.0,2.0,8.0,9.0
65533,0010000000110010,0.0,0.0,1.0,26.0,44.0,7.0,10.0,6.0
65534,0000100110000110,0.0,0.0,1.0,1.0,4.0,6.0,0.0,3.0


In [3]:
#extracting mean log fluor and stdev log fluor from fcs files
rootdir = '20200821_gate_csv/9114i'

concs = ["myc_FITC"]
FITC_min_list=[]
FITC1_bin_avg_list=[]
FITC2_bin_avg_list=[]
FITC3_bin_avg_list=[]
FITC4_bin_avg_list=[]
FITC5_bin_avg_list=[]
FITC6_bin_avg_list=[]
FITC7_bin_avg_list=[]
FITC8_bin_avg_list=[]
FITC1_bin_std_list=[]
FITC2_bin_std_list=[]
FITC3_bin_std_list=[]
FITC4_bin_std_list=[]
FITC5_bin_std_list=[]
FITC6_bin_std_list=[]
FITC7_bin_std_list=[]
FITC8_bin_std_list=[]
for conc in concs:
    #calc min PE fluor to shift all above zero
    FITC_min = 0
    FITC_list=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filepath = subdir + os.sep + file
            if conc in filepath:
                print(filepath)
                with open(filepath, encoding="utf8", errors="ignore") as csv_file:
                    csv_reader = csv_file.readlines()
                    for line in csv_reader:
                        FITC_list.append(float(line.split(",")[6]))

    if min(FITC_list) < FITC_min:
        FITC_min = min(FITC_list)
    else:
        FITC_min = FITC_min
    print("FITC_min: "+str(conc)+":"+str(FITC_min))
    FITC_min_list.append(FITC_min)
    
    #bin mean log fluor
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if conc in filepath:
                if "FITC1" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC1_bin_avg: "+str(bin_avg))
                        print(conc+"FITC1_bin_std: "+str(bin_std))
                        FITC1_bin_avg_list.append(bin_avg)
                        FITC1_bin_std_list.append(bin_std)
                elif "FITC2" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC2_bin_avg: "+str(bin_avg))
                        print(conc+"FITC2_bin_std: "+str(bin_std))
                        FITC2_bin_avg_list.append(bin_avg)
                        FITC2_bin_std_list.append(bin_std)
                elif "FITC3" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC3_bin_avg: "+str(bin_avg))
                        print(conc+"FITC3_bin_std: "+str(bin_std))
                        FITC3_bin_avg_list.append(bin_avg)
                        FITC3_bin_std_list.append(bin_std)
                elif "FITC4" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC4_bin_avg: "+str(bin_avg))
                        print(conc+"FITC4_bin_std: "+str(bin_std))
                        FITC4_bin_avg_list.append(bin_avg)
                        FITC4_bin_std_list.append(bin_std)
                elif "FITC5" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC5_bin_avg: "+str(bin_avg))
                        print(conc+"FITC5_bin_std: "+str(bin_std))
                        FITC5_bin_avg_list.append(bin_avg)
                        FITC5_bin_std_list.append(bin_std)
                elif "FITC6" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC6_bin_avg: "+str(bin_avg))
                        print(conc+"FITC6_bin_std: "+str(bin_std))
                        FITC6_bin_avg_list.append(bin_avg)
                        FITC6_bin_std_list.append(bin_std)
                elif "FITC7" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC7_bin_avg: "+str(bin_avg))
                        print(conc+"FITC7_bin_std: "+str(bin_std))
                        FITC7_bin_avg_list.append(bin_avg)
                        FITC7_bin_std_list.append(bin_std)
                elif "FITC8" in filepath:
                    print(filepath)
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        FITC_input = []
                        FITC_adj = []
                        for lines in csv_reader:
                            FITC_input.append(float(lines[6]))
                        for item in FITC_input:
                            FITC_a = item - FITC_min + 1
                            FITC_adj.append(float(FITC_a))
                        FITC_fluor = np.asarray(FITC_adj)
                        FITC_log_fluor = np.log10(FITC_fluor)
                        bin_avg = np.average(FITC_log_fluor)
                        bin_std = np.std(FITC_log_fluor)
                        print(conc+"FITC8_bin_avg: "+str(bin_avg))
                        print(conc+"FITC8_bin_std: "+str(bin_std))
                        FITC8_bin_avg_list.append(bin_avg)
                        FITC8_bin_std_list.append(bin_std)
    
with open('expression_analysis/fluor_mean_std_9114i.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(zip(concs,FITC_min_list,FITC1_bin_avg_list,FITC2_bin_avg_list,FITC3_bin_avg_list,FITC4_bin_avg_list,FITC5_bin_avg_list,FITC6_bin_avg_list,FITC7_bin_avg_list,FITC8_bin_avg_list,FITC1_bin_std_list,FITC2_bin_std_list,FITC3_bin_std_list,FITC4_bin_std_list,FITC5_bin_std_list,FITC6_bin_std_list,FITC7_bin_std_list,FITC8_bin_std_list))

20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC3.csv
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC2.csv
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC1.csv
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC5.csv
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC4.csv
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC6.csv
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC7.csv
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC8.csv
FITC_min: myc_FITC:-139.603
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC3.csv
myc_FITCFITC3_bin_avg: 2.8836138902155413
myc_FITCFITC3_bin_std: 0.15462221910280505
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC2.csv
myc_FITCFITC2_bin_avg: 2.5938764139040265
myc_FITCFITC2_bin_std: 0.051685901778764295
20200821_gate_csv/9114i/export_Sorted_9114i_myc_FITC_FITC1.csv
myc_FITCFITC1_bin_avg: 2.370871306147896
myc_FITCFITC1_bin_std: 0.1133491544174154
20200821_gate_csv/9114i/e

In [4]:
#reading in fluor values and cell counts
fluor = pd.DataFrame(pd.read_csv('expression_analysis/fluor_mean_std_9114i.csv',delimiter=',',names=['concen','FITCmin','1','2','3','4','5','6','7','8','1s','2s','3s','4s','5s','6s','7s','8s']))
cellct = pd.DataFrame(pd.read_csv('cell_counts/9114i_cellct_exp.csv',delimiter=','))

## calculate mean bin and read counts

In [5]:
concs = ['_F_']
i = 0 #index for conc
total_ct = []
bin1_cts = []
bin2_cts = []
bin3_cts = []
bin4_cts = []
bin5_cts = []
bin6_cts = []
bin7_cts = []
bin8_cts = []
for conc in concs:
    #first grab the counts and the fluor values
    #total cell counts in bin (a,b,c,d)
    Ca = float(cellct.at[i,'1'])
    Cb = float(cellct.at[i,'2'])
    Cc = float(cellct.at[i,'3'])
    Cd = float(cellct.at[i,'4'])
    Ce = float(cellct.at[i,'5'])
    Cf = float(cellct.at[i,'6'])
    Cg = float(cellct.at[i,'7'])
    Ch = float(cellct.at[i,'8'])
    bin1_ct = df_merged['counts_9114_i'+str(conc)+'1'].sum(axis=0) #sum all counts in a bin
    avg_fluor_1 = fluor.at[i, '1'] #extract mean fluor
    bin2_ct = df_merged['counts_9114_i'+str(conc)+'2'].sum(axis=0) #sum all counts in a bin
    avg_fluor_2 = fluor.at[i, '2'] #extract mean fluor
    bin3_ct = df_merged['counts_9114_i'+str(conc)+'3'].sum(axis=0) #sum all counts in a bin
    avg_fluor_3 = fluor.at[i, '3'] #extract mean fluor
    bin4_ct = df_merged['counts_9114_i'+str(conc)+'4'].sum(axis=0) #sum all counts in a bin
    avg_fluor_4 = fluor.at[i, '4'] #extract mean fluor
    bin5_ct = df_merged['counts_9114_i'+str(conc)+'5'].sum(axis=0) #sum all counts in a bin
    avg_fluor_5 = fluor.at[i, '5'] #extract mean fluor
    bin6_ct = df_merged['counts_9114_i'+str(conc)+'6'].sum(axis=0) #sum all counts in a bin
    avg_fluor_6 = fluor.at[i, '6'] #extract mean fluor
    bin7_ct = df_merged['counts_9114_i'+str(conc)+'7'].sum(axis=0) #sum all counts in a bin
    avg_fluor_7 = fluor.at[i, '7'] #extract mean fluor
    bin8_ct = df_merged['counts_9114_i'+str(conc)+'8'].sum(axis=0) #sum all counts in a bin
    avg_fluor_8 = fluor.at[i, '8'] #extract mean fluor
    total_ct_c = bin1_ct + bin2_ct + bin3_ct + bin4_ct + bin5_ct + bin6_ct + bin7_ct + bin8_ct #sum total count
    total_ct.append(total_ct_c) #append to list
    bin1_cts.append(bin1_ct)
    bin2_cts.append(bin2_ct)
    bin3_cts.append(bin3_ct)
    bin4_cts.append(bin4_ct)
    bin5_cts.append(bin5_ct)
    bin6_cts.append(bin6_ct)
    bin7_cts.append(bin7_ct)
    bin8_cts.append(bin8_ct)
    i = i + 1
    #now calc the var frequency
    df_merged['mean_bin1_%s'%conc] = (100*Ca*df_merged['counts_9114_i'+str(conc)+'1']/bin1_ct)
    df_merged['mean_bin2_%s'%conc] = (100*Cb*df_merged['counts_9114_i'+str(conc)+'2']/bin2_ct)
    df_merged['mean_bin3_%s'%conc] = (100*Cc*df_merged['counts_9114_i'+str(conc)+'3']/bin3_ct)
    df_merged['mean_bin4_%s'%conc] = (100*Cd*df_merged['counts_9114_i'+str(conc)+'4']/bin4_ct)
    df_merged['mean_bin5_%s'%conc] = (100*Ce*df_merged['counts_9114_i'+str(conc)+'5']/bin5_ct)
    df_merged['mean_bin6_%s'%conc] = (100*Cf*df_merged['counts_9114_i'+str(conc)+'6']/bin6_ct)
    df_merged['mean_bin7_%s'%conc] = (100*Cg*df_merged['counts_9114_i'+str(conc)+'7']/bin7_ct)
    df_merged['mean_bin8_%s'%conc] = (100*Ch*df_merged['counts_9114_i'+str(conc)+'8']/bin8_ct)
    #now calc the meanbin
    df_merged['meanbin_%s'%conc] = (df_merged['mean_bin1_%s'%conc] * float(avg_fluor_1) + df_merged['mean_bin2_%s'%conc] * float(avg_fluor_2) + df_merged['mean_bin3_%s'%conc] * float(avg_fluor_3) + df_merged['mean_bin4_%s'%conc] * float(avg_fluor_4) + df_merged['mean_bin5_%s'%conc] * float(avg_fluor_5) + df_merged['mean_bin6_%s'%conc] * float(avg_fluor_6) + df_merged['mean_bin7_%s'%conc] * float(avg_fluor_7) + df_merged['mean_bin8_%s'%conc] * float(avg_fluor_8))/(df_merged['mean_bin1_%s'%conc]+df_merged['mean_bin2_%s'%conc]+df_merged['mean_bin3_%s'%conc]+df_merged['mean_bin4_%s'%conc]+df_merged['mean_bin5_%s'%conc]+df_merged['mean_bin6_%s'%conc]+df_merged['mean_bin7_%s'%conc]+df_merged['mean_bin8_%s'%conc])
df_totals = pd.DataFrame(list(zip(concs, total_ct, bin1_cts, bin2_cts, bin3_cts, bin4_cts, bin5_cts, bin6_cts, bin7_cts, bin8_cts)), columns =['concen', 'total_ct', '1','2','3','4','5', '6','7','8'])
df_merged
pd.DataFrame.to_csv(df_merged,'expression_analysis/expression_meanbins_9114i.csv')

In [3]:
##merging all expression data
a = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114a.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
a.rename(columns={'meanbin__F_':'meanbin_a'},inplace=True)
b = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114b.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
b.rename(columns={'meanbin__F_':'meanbin_b'},inplace=True)
c = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114c.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
c.rename(columns={'meanbin__F_':'meanbin_c'},inplace=True)
d = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114d.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
d.rename(columns={'meanbin__F_':'meanbin_d'},inplace=True)
e = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114e.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
e.rename(columns={'meanbin__F_':'meanbin_e'},inplace=True)
f = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114f.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
f.rename(columns={'meanbin__F_':'meanbin_f'},inplace=True)
g = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114g.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
g.rename(columns={'meanbin__F_':'meanbin_g'},inplace=True)
h = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114h.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
h.rename(columns={'meanbin__F_':'meanbin_h'},inplace=True)
i = pd.DataFrame(pd.read_csv('expression_analysis/expression_meanbins_9114i.csv',delimiter=',',dtype={'variant': str}),columns=['variant','meanbin__F_'])
i.rename(columns={'meanbin__F_':'meanbin_i'},inplace=True)

In [5]:
CR9114 = pd.DataFrame()
CR9114 = pd.merge(a, b, on='variant',how='outer')
CR9114 = pd.merge(CR9114, c, on='variant',how='outer')
CR9114 = pd.merge(CR9114, d, on='variant',how='outer')
CR9114 = pd.merge(CR9114, e, on='variant',how='outer')
CR9114 = pd.merge(CR9114, f, on='variant',how='outer')
CR9114 = pd.merge(CR9114, g, on='variant',how='outer')
CR9114 = pd.merge(CR9114, h, on='variant',how='outer')
CR9114 = pd.merge(CR9114, i, on='variant',how='outer')
CR9114['exp_mean']=CR9114.mean(axis=1)
CR9114['exp_sem']=CR9114.sem(axis=1)
pd.DataFrame.to_csv(CR9114,'expression_analysis/exp_meanbin_merged.csv')